#  군집화 실습 : 고객 세그멘테이션

### 고객 세그멘테이션을 위한 고객 군집화 기준
- RFM 기법
    - Recency(R) : 가장 최근 상품 구입 일에서 오늘까지의 기간
    - Frequency(F) : 상품 구매 횟수
    - Monetary Value(M) : 총 구매 금액

## 예제 데이터 : Online Retail Data Set
https://archive.ics.uci.edu/ml/datasets/online+retail

[참고] https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
- 2019년 게시한 데이터세트로 Null 값이 없음

### 데이터 셋 로딩과 데이터 클린징

**변수**
- InvoiceNo    : 주문번호 'C'로 시작하는 것은 취소 주문      
- StockCode    : 제품 코드(Item Code)
- Description  : 제품 설명        
- Quantity     : 주문 제품 건수         
- InvoiceDate  : 주문 일자
- UnitPrice    : 제품 단가       
- CustomerID   : 고객 번호       
- Country      : 주문고객의국적

**데이터 개수 : 541909**
- CustomerID : Null 값이 13만5천 건

**데이터세트 전체 건수, 컬럼타입, Null개수 확인**

### RFM 기반 데이터 가공

**주문금액 변수 sale_amount 추가**

: 'Quantity' * 'UnitPrice'

**CustomerID 자료형을 정수형으로 변환**

**Top5 주문건수와 주문금액을 가진 고객 출력**

**'InvoiceNo'+'StockCode' 기준으로**

**R, F, M 변수**
- Recency : 가장 최근 상품 구입일에서 오늘까지의 기간
    - 'CustomerID'별로 그룹화하여 'Invoice Date' 중 가장 최근 주문일자를 사용
- Frequency : 고객별 주문건수
    - 'CustomerID'별로 그룹화하여 'InvoiceNo' 개수를 계산
- Monetary : 총구매금액
    - 'CustomerID'별로 그룹화하여 'sale_amount'의 합계를 사용

In [ ]:
# DataFrame의 groupby() 의 multiple 연산을 위해 agg() 이용
# Recency는 InvoiceDate 컬럼의 max() 에서 데이터 가공
# Frequency는 InvoiceNo 컬럼의 count() , Monetary value는 sale_amount 컬럼의 sum()


# groupby된 결과 컬럼값을 Recency, Frequency, Monetary로 변경


**Recency변수 가공**
- 오늘 날짜를 기준으로 최근 주문 일자를 뺌
- 오늘 날짜 : 2011.12.10
    - 온라인 판매 데이터가 2010.12.01~2011.12.09까지의 데이터

- 2011.12.10에서 최근주문일자를 빼고 일자 데이터(days)만 추출

### RFM 기반 고객 세그먼테이션

**R,F,M 변수 히스토그램**

**R, F, M 변수 기술통계량**

**R, M, F 변수의 표준화**

**K-평균 군집분석**

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 실루엣 계수를 면적으로 시각화한 함수 작성  
def visualize_silhouette(cluster_lists, X_features): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

In [ ]:
### 여러개의 클러스터링 갯수를 List로 입력 받아 각각의 클러스터링 결과를 시각화 
def visualize_kmeans_plot_multi(cluster_lists, X_features):
    
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    import pandas as pd
    import numpy as np
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 만큼의 sub figures를 가지는 axs 생성 
    n_cols = len(cluster_lists)
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 입력 데이터의 FEATURE가 여러개일 경우 2차원 데이터 시각화가 어려우므로 PCA 변환하여 2차원 시각화
    pca = PCA(n_components=2)
    pca_transformed = pca.fit_transform(X_features)
    dataframe = pd.DataFrame(pca_transformed, columns=['PCA1','PCA2'])
    
     # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 KMeans 클러스터링 수행하고 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링으로 클러스터링 결과를 dataframe에 저장. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(pca_transformed)
        dataframe['cluster']=cluster_labels
        
        unique_labels = np.unique(clusterer.labels_)
        markers=['o', 's', '^', 'x', '*']
       
        # 클러스터링 결과값 별로 scatter plot 으로 시각화
        for label in unique_labels:
            label_df = dataframe[dataframe['cluster']==label]
            if label == -1:
                cluster_legend = 'Noise'
            else :
                cluster_legend = 'Cluster '+str(label)           
            axs[ind].scatter(x=label_df['PCA1'], y=label_df['PCA2'], s=70,\
                        edgecolor='k', marker=markers[label], label=cluster_legend)

        axs[ind].set_title('Number of Cluster : '+ str(n_cluster))    
        axs[ind].legend(loc='upper right')
    
    plt.show()

**R, M, F 데이터의 로그변환**

In [ ]:
### Log 변환을 통해 데이터 변환

# Recency, Frequecny, Monetary 컬럼에 np.log1p() 로 Log Transformation

# Log Transformation 데이터에 StandardScaler 적용


**로그변환데이터로 K-평균 군집화 결과: 실루엣 시각화**